In [1]:
import sqlalchemy

In [2]:
import sqlalchemy as db

In [20]:
import random
from sqlalchemy import Column, Integer, Text, ForeignKey, func

In [4]:
engine = db.create_engine('sqlite:///mydb.db')

In [5]:
conn = engine.connect()

In [6]:
metadata = db.MetaData()

In [7]:
dir(db)

['ARRAY',
 'BIGINT',
 'BINARY',
 'BLANK_SCHEMA',
 'BLOB',
 'BOOLEAN',
 'BigInteger',
 'Boolean',
 'CHAR',
 'CLOB',
 'CheckConstraint',
 'Column',
 'ColumnDefault',
 'Computed',
 'Constraint',
 'DATE',
 'DATETIME',
 'DDL',
 'DECIMAL',
 'Date',
 'DateTime',
 'DefaultClause',
 'Enum',
 'FLOAT',
 'FetchedValue',
 'Float',
 'ForeignKey',
 'ForeignKeyConstraint',
 'INT',
 'INTEGER',
 'Identity',
 'Index',
 'Integer',
 'Interval',
 'JSON',
 'LABEL_STYLE_DEFAULT',
 'LABEL_STYLE_DISAMBIGUATE_ONLY',
 'LABEL_STYLE_NONE',
 'LABEL_STYLE_TABLENAME_PLUS_COL',
 'LargeBinary',
 'MetaData',
 'NCHAR',
 'NUMERIC',
 'NVARCHAR',
 'Numeric',
 'PickleType',
 'PrimaryKeyConstraint',
 'REAL',
 'SMALLINT',
 'Sequence',
 'SmallInteger',
 'String',
 'TEXT',
 'TIME',
 'TIMESTAMP',
 'Table',
 'Text',
 'ThreadLocalMetaData',
 'Time',
 'TupleType',
 'TypeDecorator',
 'Unicode',
 'UnicodeText',
 'UniqueConstraint',
 'VARBINARY',
 'VARCHAR',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__go',
 

In [8]:
users = db.Table(
    'users', metadata,
    db.Column('id', db.Integer, primary_key=True),
    db.Column('name', db.String),
    db.Column('email', db.String),
)

In [9]:
trains = db.Table(
    'trains', metadata,
    db.Column('id', db.Integer, primary_key=True),
    db.Column('name', db.String),
    db.Column('route', db.String),
)

In [10]:
tickets = db.Table(
    'tickets', metadata,
    db.Column('id', db.Integer, primary_key=True),
    db.Column('user_id', db.Integer, db.ForeignKey('users.id')),
    db.Column('train_id', db.Integer, db.ForeignKey('trains.id')),
    db.Column('way', db.Integer), # 0 - tuda 1 - tuda obratno
    db.Column('price', db.Integer),
    db.Column('departure_station', db.String),
    db.Column('arrival_station', db.String),
)

In [11]:
stations = db.Table(
    'stations', metadata,
    db.Column('id', db.Integer, primary_key=True),
    db.Column('name', db.String),
)

In [12]:
metadata.create_all(engine)

In [13]:
users_data = [{'name': f'User{i}', 'email': f'user{i}@example.com'} for i in range(1, 11)]
trains_data = [{'name': f'Train{i}', 'route': f'Route{i}'} for i in range(1, 11)]
stations_data = [{'name': f'Station{i}'} for i in range(1, 11)]

tickets_data = [
    {'user_id': i, 'train_id': i, 'way': random.randint(0, 1),'price': i * random.randint(1, 10),
     'departure_station': f'Station{random.randint(1, 10)}', 'arrival_station': f'Station{random.randint(1, 10)}'}
    for i in range(1, 11)
]

In [14]:
insert_users = users.insert().values(users_data)
insert_trains = trains.insert().values(trains_data)
insert_stations = stations.insert().values(stations_data)
insert_tickets = tickets.insert().values(tickets_data)

In [15]:
conn.execute(insert_users)
conn.execute(insert_trains)
conn.execute(insert_stations)
conn.execute(insert_tickets)

In [16]:
all_users = conn.execute(users.select()).fetchall()
all_trains = conn.execute(trains.select()).fetchall()
all_stations = conn.execute(stations.select()).fetchall()
all_tickets = conn.execute(tickets.select()).fetchall()

In [17]:
print("Users:")
for user in all_users:
    print(f"ID: {user.id}, Name: {user.name}, Email: {user.email}")

print("\nTrains:")
for train in all_trains:
    print(f"ID: {train.id}, Name: {train.name}, Route: {train.route}")

print("\nStations:")
for station in all_stations:
    print(f"ID: {station.id}, Name: {station.name}")

print("\nTickets:")
for ticket in all_tickets:
    print(f"ID: {ticket.id}, User ID: {ticket.user_id},  Train ID: {ticket.train_id}, Way: {ticket.way}, Price: {ticket.price}, "
          f"Departure Station: {ticket.departure_station}, Arrival Station: {ticket.arrival_station}")


Users:
ID: 1, Name: User1, Email: user1@example.com
ID: 2, Name: User2, Email: user2@example.com
ID: 3, Name: User3, Email: user3@example.com
ID: 4, Name: User4, Email: user4@example.com
ID: 5, Name: User5, Email: user5@example.com
ID: 6, Name: User6, Email: user6@example.com
ID: 7, Name: User7, Email: user7@example.com
ID: 8, Name: User8, Email: user8@example.com
ID: 9, Name: User9, Email: user9@example.com
ID: 10, Name: User10, Email: user10@example.com

Trains:
ID: 1, Name: Train1, Route: Route1
ID: 2, Name: Train2, Route: Route2
ID: 3, Name: Train3, Route: Route3
ID: 4, Name: Train4, Route: Route4
ID: 5, Name: Train5, Route: Route5
ID: 6, Name: Train6, Route: Route6
ID: 7, Name: Train7, Route: Route7
ID: 8, Name: Train8, Route: Route8
ID: 9, Name: Train9, Route: Route9
ID: 10, Name: Train10, Route: Route10

Stations:
ID: 1, Name: Station1
ID: 2, Name: Station2
ID: 3, Name: Station3
ID: 4, Name: Station4
ID: 5, Name: Station5
ID: 6, Name: Station6
ID: 7, Name: Station7
ID: 8, Name: 

In [25]:
# having
select_having = conn.execute(db.select(tickets.c.price, func.count(tickets.c.user_id)).group_by(tickets.c.way).having(tickets.c.price > 10 ))
result = select_having.fetchall()
print(result)

[(27, 7)]


In [ ]:
#union
select_users_trains = conn.execute(db.select(users.c.name, users.c.email).union(db.select(trains.c.name, trains.c.route)))
result = select_users_trains.fetchall()
print(result)

In [30]:
# inner join
select_join = conn.execute(db.select(trains.c.name, tickets.c.departure_station).select_from(trains.join(tickets, trains.c.id == tickets.c.train_id))).all()
result = select_join
print(result)

[('Train1', 'Station3'), ('Train2', 'Station5'), ('Train3', 'Station10'), ('Train4', 'Station5'), ('Train5', 'Station10'), ('Train6', 'Station3'), ('Train7', 'Station9'), ('Train8', 'Station5'), ('Train9', 'Station6'), ('Train10', 'Station4')]


In [ ]:
conn.close()